In [ ]:
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'none'

%matplotlib inline

# Scikit-Image
Dans cette section, nous allons présenter les possibilités offertes par la librairie *scikit-image*. Cette dernière n'étant pas exhaustive, nous vous renvoyons donc sur la page principale de *scikit-image* pour plus d'informations:
http://scikit-image.org/

Cette section est librement basée sur les tutoriels proposés par `scikit-image`:
https://github.com/scikit-image/skimage-tutorials

## 1. Opérations de base
### 1.1 Quelques généralités 
Dans `skimage`, les images sont gérées par des `numpy.ndarray`. 

In [ ]:
from skimage import data

coins = data.coins()

print(type(coins), coins.dtype, coins.shape)
plt.imshow(coins, cmap='gray', interpolation='nearest');

In [ ]:
type(coins)

De base les images sont représentées par des entiers.

In [ ]:
cat = data.chelsea()
print("Shape:", cat.shape)
print("Values min/max:", cat.min(), cat.max())

plt.imshow(cat, interpolation='nearest');

Les fonctions `img_as_float` et `img_as_ubyte` permettent de les convertir en float ou en int.

In [ ]:
from skimage import img_as_float, img_as_ubyte

image = data.chelsea()

image_float = img_as_float(image)
image_ubyte = img_as_ubyte(image)

print("type, min, max:", image_float.dtype, image_float.min(), image_float.max())
print("type, min, max:", image_ubyte.dtype, image_ubyte.min(), image_ubyte.max())

print("231/255 =", 231/255.)

Pour éviter tout problème, skimage conseille de mettre son code sous cette forme :

```python
def my_function(any_image):
   float_image = img_as_float(any_image)
   # Proceed, knowing image is in [0, 1]
```

Il est recommandé d'utiliser la représentation flottante dans la mesure où car c'est celle qui est utilisée par défaut dans `scikit-image`.

### 1.2 Accesseurs
Comme toute les images sont gérées par `numpy`, il est possible d'accéder aux pixels en utilisant les accesseurs donnés par `numpy`.

In [ ]:
cat[10:110, 10:110, :] = [255, 0, 0]  # [red, green, blue]
plt.imshow(cat);

Nous pouvons définir une version sous-échantillonnées de l'image de la manière suivante :

In [ ]:
image = data.camera()
pixelated = image[::10, ::10]
plt.imshow(pixelated);

Il est d'ailleurs possible d'utiliser `matplotlib` pour afficher les images avec la fonction `imshow`.
L'attribut `cmap` permet d'utiliser différentes carte de couleurs pour les afficher.

In [ ]:
image = data.camera()
face = image[80:160, 200:280]

fig, (ax_jet, ax_viridis, ax_gray) = plt.subplots(ncols=3, figsize=(10, 10))
ax_jet.imshow(face, cmap='jet')
ax_viridis.imshow(face, cmap='viridis')
ax_gray.imshow(face, cmap='gray');

Je vous renvoie vers cette [video](https://www.youtube.com/watch?v=xAoljeRJ3lU) pour une explication de `viridis` et de sa comparaison face à `jet`.

### Exercice
À partir de l'image `data.camera()`, créer un masque de sorte que les niveaux de gris **inférieur à 87 soient blancs** et que **les pixels en dehors d'un cercle centré et du même diamètre que l'image soient noirs**.

Vous aurez besoin de la fonction `np.ogrid` pour créer le cercle.

In [ ]:
camera = data.camera()
mask = camera < 87
camera[mask] = 255

l_x, l_y = camera.shape[0], camera.shape[1]
X, Y = np.ogrid[:l_x, :l_y]
outer_disk_mask = (X - l_x / 2)**2 + (Y - l_y / 2)**2 > (l_x / 2)**2
camera[outer_disk_mask] = 0

plt.imshow(camera);

### 1.3 Lecture / Ecriture
`Skimage` propose différentes fonctions pour lire/écrire des images : 

In [ ]:
from skimage import io

im = io.imread('exo_img/skimage/input/salta_1.png')
plt.imshow(image);

In [ ]:
io.imsave('exo_img/skimage/input/salta_copy.jpg', im)

`ImageCollection` permet de lire un ensemble d'images.

In [ ]:
pano_imgs = io.ImageCollection('exo_img/skimage/input/pano/JDW_03*')

Nous introduisons ici une fonction pour comparer plusieurs images entre elles. Cette dernière nous sera utile par la suite de ce notebook.

In [ ]:
def compare(*images, **kwargs):
    """
    Utility function to display images side by side.
    
    Parameters
    ----------
    image0, image1, image2, ... : ndarrray
        Images to display.
    labels : list
        Labels for the different images.
    """
    f, axes = plt.subplots(1, len(images), **kwargs)
    axes = np.array(axes, ndmin=1)
    
    labels = kwargs.pop('labels', None)
    if labels is None:
        labels = [''] * len(images)
    
    for n, (image, label) in enumerate(zip(images, labels)):
        axes[n].imshow(image, interpolation='nearest', cmap='gray')
        axes[n].set_title(label)
        axes[n].axis('off')
    
    f.tight_layout()

In [ ]:
compare(*pano_imgs, figsize=(12, 10))

## 2. Filtrage
Plusieurs filtres simples sont proposés dans le module `skimage.filters`:

In [ ]:
from skimage import filters

Application d'un filtre TV:

In [ ]:
from skimage.restoration import denoise_tv_bregman

denoised = denoise_tv_bregman(pixelated, 4)
io.imshow(denoised);

Un filtre de détecteur de contour (Sobel):

In [ ]:
pixelated_gradient = filters.sobel(pixelated)
io.imshow(pixelated_gradient);

Ou encore un filtre médian :

In [ ]:
from skimage.morphology import disk
selem = np.float64(disk(2))

median = filters.rank.median(pixelated, selem)
io.imshow(median);

Néanmoins ces filtres sont généralement définis uniquement pour des niveaux de gris. 

La librairie propose le décorateur `@adapt_rgb` pour généraliser son utilisation aux images couleur. 

In [ ]:
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value

Le décorateur se place avant la fonction. L'attribut définit la manière dont doit être appliqué le filtre sur les trois canaux. 
- `each_channel` applique indépendament le filtre sur chaque canal couleur. 
- `hsv_value` convertit l'image dans l'espace couleurs HSV et applique le filtre sur chaque canal indépendament.

In [ ]:
@adapt_rgb(each_channel)
def sobel_each(image):
    return filters.sobel(image)


@adapt_rgb(hsv_value)
def sobel_hsv(image):
    return filters.sobel(image)

In [ ]:
from skimage.exposure import rescale_intensity

image = data.astronaut()

fig = plt.figure(figsize=(14, 7))
ax_each = fig.add_subplot(121, adjustable='box-forced')
ax_hsv = fig.add_subplot(122, sharex=ax_each, sharey=ax_each,
                         adjustable='box-forced')

ax_each.set_title("Sobel filter computed\n on individual RGB channels")
ax_each.imshow(rescale_intensity(1 - sobel_each(image)));

ax_hsv.set_title("Sobel filter computed\n on (V)alue converted image (HSV)")
ax_hsv.imshow(rescale_intensity(1 - sobel_hsv(image)));

Il est possible de définir sa propre convertion de couleurs en définissant une fonction de convertion. 

Cette dernière doit être de la forme suivante :
``` python
def as_personal_convertion(image_filter, image, *args, **kwargs):
    # Tout un tas d'opération pour appliquer image_filter a l'image
    ...
    return filtered_image
```

Il est également possible de généraliser un traitement à un ensemble d'images en utilisant les possibilités offertes par Pythons.

In [ ]:
from skimage.color import rgb2gray
pano0, pano1, pano2 = [rgb2gray(im) for im in pano_imgs]

In [ ]:
compare(pano0, pano1, pano2, figsize=(12, 10))

Il est tout à fait possible de définir ses propres filtres à partir des `numpy.array`. Considérer l'image de test suivante :

In [ ]:
bright_square = np.zeros((7, 7), dtype=float)
bright_square[2:5, 2:5] = 1
print(bright_square)

In [ ]:
plt.imshow(bright_square, cmap='gray');

Nous définissons le filtre moyenneur de la manière suivante :

In [ ]:
mean_kernel = 1.0/9.0 * np.ones((3, 3))

%precision 2
print(mean_kernel)

Pour l'appliquer, nous allons utiliser la fonction convolve fourni par `scipy.ndimage` :

In [ ]:
from scipy.ndimage import convolve
convolve?

Le résultat obtenue après convolution de `bright square`:

In [ ]:
%precision 2
print(convolve(bright_square, mean_kernel))

Le résultat obtenu sur une image:

In [ ]:
new_image = convolve(pixelated, mean_kernel)
plt.imshow(new_image);

### Exercice
- Définissez un filtre qui calcule le gradient de l'image.
- Ajouter un décorateur de sorte que si l'image est couleur, elle soit convertie en niveau de gris avant de calculer le gradient de l'image.

In [ ]:
from skimage.color import rgb2gray

def as_gray(image_filter, image, *args, **kwargs):
    gray_image = rgb2gray(image)
    return image_filter(gray_image, *args, **kwargs)

In [ ]:
@adapt_rgb(as_gray)
def compute_gradient(im):
    im = img_as_float(im)
    
    # Replace the kernels below with your difference filter
    # `ones` is used just for demonstration and your kernel 
    # should be larger than (1, 1)
    vertical_edge_kernel = np.array([[-1, 1]])
    horizontal_edge_kernel = np.array([[-1], [1]])

    # As discussed earlier, you may want to replace pixelated
    # with a different image.
    image = pixelated
    # NOTE: A **vertical** gradient has a strong response at 
    # **horizontal** edges and vice versa.
    vertical_gradient = convolve(im, horizontal_edge_kernel)
    horizontal_gradient = convolve(im, vertical_edge_kernel)
    
    return vertical_gradient, horizontal_gradient

Nous vous proposons de vérifier que les résultats sont corrects :

In [ ]:
v_img, h_img = compute_gradient(data.astronaut())

plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(v_img, cmap='viridis');

plt.subplot(122)
plt.imshow(h_img, cmap='viridis');

## 3. Retouche
La librairie fournie également quelques fonctions de retouches des images contenues dans le module `skimage.exposure`. En voici quelques exemples :

In [ ]:
from skimage import exposure

Une fonction pour ajuster le niveau gamma de l'image:

In [ ]:
exposure.exposure.adjust_gamma?

In [ ]:
im = data.moon()
gamma_img = exposure.exposure.adjust_gamma(im, 1.)
plt.imshow(gamma_img);

Une autre qui permet d'ajuster les variations d'histogrammes.

In [ ]:
exposure.equalize_adapthist?

In [ ]:
log_img = exposure.equalize_adapthist(im, clip_limit=0.03)
plt.imshow(log_img);

### Exercice
Nous vous proposons de programmer la version généralisée du `Midway Equilization Histogram` proposée par Julie Delon. Cet algorithme permet de limiter les variations de luminosité et de contraste d'une même série d'images.

Il fonctionne de la manière suivante :
- Calculer les histogrammes cumulés des deux images en utilisant `exposure.cumularive_distribution`
- Trouver le niveau gris $n_2$ de l'image 2 qui possèdent le même distribution cumulées que le niveau de gris $n_1$  requête de l'image 1.
- Définir le nouveaux niveaux gris par $n_m = (n_1 + n_2) / 2$
- Mettre à jour $n_1 = n_m, n_2 = n_m$.

In [ ]:
def apply_midway_general(u_1, u_2):
    u_1 = img_as_float(u_1)
    u_2 = img_as_float(u_2)
    
    cum_hist1, _ = exposure.cumulative_distribution(u_1.ravel())
    cum_hist2, _ = exposure.cumulative_distribution(u_2.ravel())

    f_12 = (np.arange(256) + np.array([np.argmin(h1 > cum_hist2)
                                       for h1 in cum_hist1])) / 510.
    f_21 = (np.arange(256) + np.array([np.argmin(h2 > cum_hist1)
                                       for h2 in cum_hist2])) / 510.

    u_midway_1 = f_12[np.int_(255. * u_1.ravel())]
    u_midway_2 = f_21[np.int_(255. * u_2.ravel())]

    return u_midway_1.reshape(u_1.shape), u_midway_2.reshape(u_2.shape)

Vous pouvez contrôler les résultats avec les lignes suivantes:

In [ ]:
im_1 = io.imread('exo_img/skimage/input/fitz_roy_1.png')
im_2 = io.imread('exo_img/skimage/input/fitz_roy_2.png')

midway_1, midway_2 = apply_midway_general(im_1, im_2)

plt.figure(figsize=(17, 10))
plt.subplot(221)
plt.axis('off')
plt.imshow(im_1)
plt.title('Image 1 avant correction')

plt.subplot(222)
plt.axis('off')
plt.imshow(im_2)
plt.title('Image 2 avant correction')

plt.subplot(223)
plt.axis('off')
plt.imshow(midway_1)
plt.title('Image 1 apres correction')

plt.subplot(224)
plt.axis('off')
plt.imshow(midway_2)
plt.title('Image 2 apres correction')

plt.show()

## 4. Travailler avec des blocs
Les blocs permettent de diviser l'image en sous imagettes **sans aucun recouvrement**. Attention ceci est différent de la notion de patches.

In [ ]:
from skimage.util.shape import view_as_blocks

In [ ]:
view_as_blocks(np.reshape(np.arange(16), (4, 4)), (2, 2)).shape

In [ ]:
l = rgb2gray(data.astronaut())
block_shape = (4, 4)
view = view_as_blocks(l, block_shape)

In [ ]:
view.shape

In [ ]:
flatten_view = view.reshape(view.shape[0], view.shape[1], -1)

In [ ]:
flatten_view.shape

A partir de ses derniers il est néanmoins possible de définir un certain nombre de traitement des images :

In [ ]:
mean_view = np.mean(flatten_view, axis=2)

plt.imshow(mean_view);

In [ ]:
max_view = np.max(flatten_view, axis=2)

plt.imshow(max_view);

In [ ]:
median_view = np.median(flatten_view, axis=2)

plt.imshow(median_view);

Pour extraire des patches, nous vous renvoyons vers la fonction `view_as_windows` qui permet d'extraire des sous parties d'une image avec un recouvrement.

**Exercice Bonus :** Ecrivez une version simple du filtre non-local à partir de `view-as-windows`.

## 5. Extraction de features
La libraire offre également un certain nombre de fonction permettant l'extraction et l'appariement des features.

### 5.1 Détection de features
Nous allons ici utiliser l'`ORB` (d'autres sont disponibles dans la librairie). Je vous renvoie vers la documentation de `skimage` pour plus d'informations.

In [ ]:
from skimage.feature import ORB

In [ ]:
ORB?

In [ ]:
orb = ORB(n_keypoints=800, fast_threshold=0.05)

In [ ]:
orb.detect_and_extract(pano0)
keypoints0 = orb.keypoints
descriptors0 = orb.descriptors

In [ ]:
orb.detect_and_extract(pano1)
keypoints1 = orb.keypoints
descriptors1 = orb.descriptors

### 5.2 Appariement des features
La librairie possède également une fonction d'appariement des features :

In [ ]:
from skimage.feature import match_descriptors

In [ ]:
match_descriptors?

In [ ]:
matches01 = match_descriptors(descriptors0, descriptors1, cross_check=True)

In [ ]:
from skimage.feature import plot_matches
fig, ax = plt.subplots(1, 1, figsize=(15, 12))

plot_matches(ax, pano0, pano1, keypoints0, keypoints1, matches01)
ax.axis('off');

### 5.3 Exercice : Appariement robuste
Comme nous l'avons vu précédemment, la fonction appariement peut avoir quelques ratés. Par conséquent, nous vous proposant d'utiliser la méthode `RANSAC` pour faire un appariement plus sur.

In [ ]:
from skimage.measure import ransac

In [ ]:
ransac?

In [ ]:
from skimage.transform import ProjectiveTransform

In [ ]:
ProjectiveTransform?

Utiliser la méthode `RANSAC` pour obtenir un appariement robuste:

In [ ]:
# Select keypoints from 
#   * source (image to be registered): pano0
#   * target (reference image): pano1, our middle frame registration target
src = keypoints0[matches01[:, 0]][:, ::-1]
dst = keypoints1[matches01[:, 1]][:, ::-1]

model_robust01, inliers01 = ransac((src, dst), ProjectiveTransform,
                                   min_samples=4, residual_threshold=1, max_trials=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 12))
plot_matches(ax, pano0, pano1, keypoints0, keypoints1, matches01[inliers01])
ax.axis('off');

## 6. Warping
Maintenant que certaines features sont mises en correspondances, on va pouvoir corriger la déformation des images et ainsi définir un panorama. Commençons par déterminer la taille de l'image résultante :

In [ ]:
from skimage.transform import SimilarityTransform

# Shape of middle image, our registration target
r, c = pano1.shape[:2]

# Note that transformations take coordinates in (x, y) format,
# not (row, column), in order to be consistent with most literature
corners = np.array([[0, 0],
                    [0, r],
                    [c, 0],
                    [c, r]])

# Warp the image corners to their new positions
warped_corners01 = model_robust01(corners)

# Find the extents of both the reference image and the warped
# target image
all_corners = np.vstack((warped_corners01, corners))

# The overall output shape will be max - min
corner_min = np.min(all_corners, axis=0)
corner_max = np.max(all_corners, axis=0)
output_shape = (corner_max - corner_min)

# Ensure integer shape with np.ceil and dtype conversion
output_shape = np.ceil(output_shape[::-1]).astype(int)

Nous pouvons appliquer une homomgraphie sur l'une des deux images de sorte à ce qu'elle puisse être associé ensemble :

In [ ]:
from skimage.transform import warp

# This in-plane offset is the only necessary transformation for the middle image
offset1 = SimilarityTransform(translation= -corner_min)

# Translate pano1 into place
pano1_warped = warp(pano1, offset1.inverse, order=3,
                    output_shape=output_shape, cval=-1)

# Acquire the image mask for later use
pano1_mask = (pano1_warped != -1)  # Mask == 1 inside image
pano1_warped[~pano1_mask] = 0      # Return background values to 0

Le masque précédent est utilie pour ne pas faire une moyenne avec les pixels noirs :

In [ ]:
# Warp pano0 (left) to pano1
transform01 = (model_robust01 + offset1).inverse
pano0_warped = warp(pano0, transform01, order=3,
                    output_shape=output_shape, cval=-1)

pano0_mask = (pano0_warped != -1)  # Mask == 1 inside image
pano0_warped[~pano0_mask] = 0      # Return background values to 0

In [ ]:
compare(pano0_warped, pano1_warped, figsize=(12, 10));

###  7. Exercice : Panorama
Les deux images sont désormais rectifiées, il ne reste plus qu'à les assembler sur une même image :

In [ ]:
# Add the three images together. This could create dtype overflows!
# We know they are are floating point images after warping, so it's OK.
merged = (pano0_warped + pano1_warped)

In [ ]:
# Track the overlap by adding the masks together
overlap = (pano0_mask * 1.0 +  # Multiply by 1.0 for bool -> float conversion
           pano1_mask)

In [ ]:
# Normalize through division by `overlap` - but ensure the minimum is 1
normalized = merged / np.maximum(overlap, 1)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

ax.imshow(normalized, cmap='gray')

plt.tight_layout()
ax.axis('off');

Le résultat précédent n'est pas parfait mais permet d'avoir une première approche du problème du panorama. Pour un résultat optimal, nous devrions appliquer un algorithme qui calcule automatiquement les bons masques à appliquer sur les deux images de sorte à ce que les divisions ne soient plus apparentes. 

Si tout ceci vous intéresse, nous vous renvoyons vers le tutoriel panorama de `Skimage` à `Scipy 2015` : 
https://github.com/scikit-image/skimage-tutorials